In [1]:
import cv2 as cv
import numpy as np
import pandas as pd
import PIL
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from evaluation.vlc import VLC
from utils.path_utils import Files
from evaluation.simulator import TimingConfig
from evaluation.analysis import Plotter

%matplotlib qt
pd.options.display.max_columns = 30

In [2]:
from dataset.bbox_utils import *
import numpy as np
delta = np.array([[2],[4]])
boxes = np.ones((3,4))
boxes[:,0] = range(3)
boxes[:,1] = [1,2,4]

center = BoxUtils.center(boxes)
print(center)
np.diff(center, n=1, axis=0) / delta


[[0.5 1.5]
 [1.5 2.5]
 [2.5 4.5]]


array([[0.5 , 0.5 ],
       [0.25, 0.5 ]])

# Analysis

## VLC

In [3]:

# directory = "/Users/guycohen/Desktop/Bio-Project/eval/micro"
# files = Files(directory, extension='png', sorting_key=lambda f: int(f.split('_')[-1].split('.')[0]))
# timing_config = TimingConfig.load_json("/Users/guycohen/Desktop/Bio-Project/time_config.json")
# log_path = "/Users/guycohen/Desktop/Bio-Project/eval/bboxes2.csv"


In [4]:
# vlc = VLC(files, timing_config, log_path, 'mic')
# vlc.mainloop()

## Plots

In [5]:
def cond(df: pd.DataFrame, threshold: float=1.5) -> pd.DataFrame:
    mask1 = (df['cycle_step'] == 14) & (df['worm_center_dist'] > threshold)
    cycles = df[mask1]['cycle'].unique()
    return df['cycle'].isin(cycles)

def cond_imaging(df:pd.DataFrame) -> pd.DataFrame:
    return df['phase'] == 'imaging'

def rolling_avg(data, col, window_size=5):
    data[col+'_avg'] = Plotter.rolling_average(data, window_size=window_size, column=col)
    return data

In [6]:

# original_log_path = "/Users/guycohen/Desktop/Bio-Project/eval/bboxes2.csv"
# log_path = "/Users/guycohen/Desktop/Bio-Project/eval_corrected_movement/bboxes.csv"
# log_path = "/Users/guycohen/Desktop/Bio-Project/Optimal/bboxes.csv"
log_path = "/Users/guycohen/Desktop/Bio-Project/Controller2/bboxes.csv"
timing_config = TimingConfig.load_json("/Users/guycohen/Desktop/Bio-Project/time_config.json")

pltr = Plotter(log_path, timing_config)
# pltr_orig = Plotter(original_log_path, timing_config)

In [7]:
data = pltr.print_statistics(n=10)
# data_orig = pltr_orig.print_statistics()

##################### No Preds #####################
Num of No Preds: 3
No prediction in frames: [21997, 22004, 61199]
corresponding cycle steps: [7, 14, 14]
##################### Cycles #####################
Num of cycles: 4080
##################### Area Diff #####################
Non Perfect Predictions: 37.254%
##################### General #####################


,frame,cycle,plt_x,plt_y,cam_x,cam_y,cam_w,cam_h,mic_x,mic_y,mic_w,mic_h,wrm_x,wrm_y,wrm_w,wrm_h,cycle_step,wrm_center_x,wrm_center_y,mic_center_x,mic_center_y,wrm_speed,wrm_speed_x,wrm_speed_y,bbox_area_diff,bbox_edge_diff,worm_angle,worm_deviation_x,worm_deviation_y,worm_deviation
count,61182.000000,61182.000000,61182.000000,61182.000000,61182.000000,61182.000000,61182.0,61182.0,61182.000000,61182.000000,61182.0,61182.0,61182.000000,61182.000000,61182.000000,61182.000000,61182.000000,61182.000000,61182.000000,61182.000000,61182.000000,61182.000000,61182.000000,61182.000000,6.118200e+04,61182.000000,61182.000000,61182.000000,61182.000000,61182.000000
mean,30606.781325,2039.985437,1266.047743,1121.192622,1266.047743,1121.192622,360.0,360.0,1436.047743,1291.192622,20.0,20.0,1440.064808,1294.509149,12.050882,13.216355,6.999771,1446.090249,1301.117326,1446.047743,1301.192622,0.207497,0.003301,-0.006391,5.019177e-02,0.750182,-0.081822,0.042506,-0.075296,2.596267
std,17662.663921,1177.510902,192.782035,215.124933,192.782035,215.124933,0.0,0.0,192.782035,215.124933,0.0,0.0,192.955573,215.319167,3.071914,3.212946,4.320450,192.808859,215.153416,192.782035,215.124933,0.176863,0.183380,0.201634,1.049594e-01,1.503968,1.778381,2.314477,2.535469,2.247703
min,15.000000,1.000000,885.000000,541.000000,885.000000,541.000000,360.0,360.0,1055.000000,711.000000,20.0,20.0,1055.408157,712.498398,5.599228,5.907318,0.000000,1063.626976,720.871139,1065.000000,721.000000,0.000042,-1.344165,-1.366954,-2.349879e-14,0.000000,-3.119770,-18.903076,-19.031426,0.001102
25%,15310.250000,1020.000000,1149.000000,1028.000000,1149.000000,1028.000000,360.0,360.0,1319.000000,1198.000000,20.0,20.0,1323.442200,1200.018482,9.541931,10.766243,3.000000,1329.225451,1207.887367,1329.000000,1208.000000,0.067227,-0.081267,-0.104939,-2.278495e-15,0.000000,-1.718033,-0.974853,-1.238764,0.842819
50%,30607.500000,2040.000000,1168.000000,1180.000000,1168.000000,1180.000000,360.0,360.0,1338.000000,1350.000000,20.0,20.0,1341.224594,1351.891373,11.856727,14.264656,7.000000,1348.435383,1359.523685,1348.000000,1360.000000,0.174804,0.000382,-0.003461,7.166434e-15,0.000000,-0.169764,0.019516,-0.080544,2.060195
75%,45902.750000,3060.000000,1490.000000,1260.000000,1490.000000,1260.000000,360.0,360.0,1660.000000,1430.000000,20.0,20.0,1663.674500,1434.789707,14.520523,15.803734,11.000000,1669.568335,1440.430336,1670.000000,1440.000000,0.299967,0.082204,0.087413,5.758979e-02,0.889996,1.402337,0.989445,1.049888,3.664127
max,61198.000000,4079.000000,1549.000000,1389.000000,1549.000000,1389.000000,360.0,360.0,1719.000000,1559.000000,20.0,20.0,1726.091202,1560.643234,19.665741,19.883560,14.000000,1729.797531,1569.973503,1729.000000,1569.000000,1.596883,1.032538,1.590397,1.000000e+00,17.423416,3.141593,15.599251,18.332169,21.259516


In [ ]:
(data['mic_center_x'] - data['wrm_center_x']).hist(bins=15)

In [ ]:
sns.scatterplot(data=data, x='frame', y='worm_deviation', hue='wrm_speed')

In [17]:
def plot_histogram(pltr, x_col:str, n:int=1, hue=None, condition=None, **kwargs):
    data = pltr.data_prep_frames(n=n)
    
    if condition is not None:
            data = data[condition(data)]
    fig, ax = plt.subplots()
    sns.histplot(data=data, x=x_col, hue=hue,stat='density', **kwargs)

def plot_jointplot(pltr, x_col:str, y_col:str, n:int=1, kind:str='scatter', hue=None, condition=None, transform=None, **kwargs):
    data = pltr.data_prep_frames(n=n)
    
    if transform is not None:
        data = transform(data)
        
    if condition is not None:
            data = data[condition(data)]
    
    sns.jointplot(data=data, x=x_col, y=y_col, hue=hue, kind=kind, **kwargs)

# plot_histogram(pltr, 'worm_deviation', n=15, hue='worm_deviation')

In [ ]:
plot_jointplot(pltr, 'wrm_speed', 'worm_deviation', n=5, kind='scatter')

In [ ]:
plot_jointplot(pltr, x_col='wrm_speed_x', y_col='wrm_speed_y', n=5, kind='scatter', hue='worm_angle_avg', transform=lambda data: rolling_avg(data, 'worm_angle', window_size=5))

In [18]:
plot_histogram(pltr, x_col='bbox_area_diff', condition=lambda df: df['bbox_area_diff'] > 1e-3)

/Users/guycohen/opt/anaconda3/envs/bio-proj/lib/python3.12/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):


In [ ]:
def avg_speed(data:pd.DataFrame, n_list:list[int]=[10]) -> np.ndarray:
    data = data.copy()
    
    wrm_speed_x = np.zeros(len(data['frame']))
    wrm_speed_y = np.zeros(len(data['frame']))
    for n in n_list:
        wrm_speed_x += data["wrm_center_x"].diff(n) / len(n_list)
        wrm_speed_y += data["wrm_center_y"].diff(n) / len(n_list)
        # avgs.append(Plotter.worm_angle(data, n=n))
    # mean of all averages
    data['wrm_speed_x_avg'] = wrm_speed_x
    data['wrm_speed_y_avg'] = wrm_speed_y
    return data

def worm_angle(data:pd.DataFrame, n:int=15) -> np.ndarray:
    data = data.copy()

    wrm_speed_x = data["wrm_center_x"].diff(n) / n
    wrm_speed_y = data["wrm_center_y"].diff(n) / n
    angle_speed = np.arctan2(wrm_speed_y, wrm_speed_x)

    min_h, min_w = np.min(data["wrm_h"]), np.min(data["wrm_w"])
    x_sign = np.sign(data["wrm_center_x"].diff(3))
    y_sign = np.sign(data["wrm_center_y"].diff(3))
    # data['wrm_speed_x_avg'] = (data['wrm_w']-min_w)*x_sign
    # data['wrm_speed_y_avg'] = (data['wrm_h']-min_h)*y_sign
    angle = np.arctan2((data['wrm_h']-min_h)*y_sign, (data['wrm_w']-min_w)*x_sign)

    angle_diff = np.abs(angle - angle_speed) / (2*np.pi)
    data['wrm_speed_x_avg'] = wrm_speed_x * (1-angle_diff)
    data['wrm_speed_y_avg'] = wrm_speed_y * (1-angle_diff)
    return data

In [ ]:
def plot_flow(pltr, x_col:str, y_col:str, n:int=1, kind:str='scatter', hue=None, condition=None, transform=None, **kwargs):
    data = pltr.data_prep_frames(n=n)
    
    if transform is not None:
        data = transform(data)
        
    if condition is not None:
            data = data[condition(data)]
    
    # cmap = sns.color_palette("husl", as_cmap=True, n_colors=len(data['frame']))
    fig, ax = plt.subplots()
    plt.quiver(data[x_col], data[y_col], data['wrm_speed_x_avg'], data['wrm_speed_y_avg'], np.linspace(0,1,len(data['frame'])), **kwargs)
    ax.invert_yaxis()
    ax.grid()


# plot_flow(pltr, x_col='wrm_center_x', y_col='wrm_center_y', n=15, angles='xy', transform=lambda data: rolling_avg(data, 'worm_angle', window_size=1))
plot_flow(pltr, x_col='wrm_center_x', y_col='wrm_center_y', n=5, angles='xy', scale_units='xy', scale=1,
          transform=lambda df: worm_angle(df, 15), 
        #   condition=lambda df: df['cycle_step'] == 9,
          )

In [ ]:
# cols = ['frame', 'cycle', 'plt_x', 'plt_y', 'cam_x', 'cam_y', 'cam_w',
#        'cam_h', 'mic_x', 'mic_y', 'mic_w', 'mic_h', 'wrm_x', 'wrm_y', 'wrm_w',
#        'wrm_h', 'wrm_center_x', 'wrm_center_y', 'mic_center_x', 'mic_center_y',
#        'wrm_speed', 'bbox_area_diff', 'bbox_edge_diff']


# (data[cols] - data_orig[cols]).describe()

In [ ]:
data.columns

In [ ]:

pltr.plot_2d_deviation(n=2, hue='cycle_step')

In [ ]:
pltr.plot_deviation()

In [ ]:
pltr.plot_area_vs_speed_guy(n=1, window_size=1, hue='phase')

In [ ]:
pltr.plot_cycle_step_vs_speed()

In [ ]:
pltr.plot_trajectory(n=15, hue='wrm_speed')

In [ ]:
def plot_speed_vs_time(pltr, n: int = 1, window_size: int = 15, hue=None, condition=None) -> plt.Figure:
        data = pltr.data_prep_frames(n=n)
        data["wrm_speed_avg"] = Plotter.rolling_average(data, window_size=window_size, column="wrm_speed")
        fig, ax = plt.subplots()
        if condition is not None:
            mask = condition(data) & mask
        g = sns.histplot(data=data, x="wrm_speed", ax=ax, hue=hue)
        g.figure.suptitle(f"n = {n}, rolling window = {window_size}")
        return g.figure

In [ ]:
plot_speed_vs_time(pltr, n=15, window_size=1, hue='phase')

In [ ]:
timing_config.mm_per_px / timing_config.ms_per_frame

In [ ]:
def plot_2d_deviation_with_angle(pltr, n:int=1, hue="cycle_step", condition=None) -> plt.Figure:
        data = pltr.data_prep_frames(n=n)
        data["worm_center_dist_x"] = data["wrm_center_x"] - data["mic_center_x"]
        data["worm_center_dist_y"] = data["wrm_center_y"] - data["mic_center_y"]

        data["cycle_step"] = data["frame"] % pltr.timing_config.cycle_length
        min_h, min_w = np.min(data["wrm_h"]), np.min(data["wrm_w"])
        x_sign = np.sign(data["wrm_center_x"].diff(n))
        y_sign = np.sign(data["wrm_center_y"].diff(n))
        data['angle'] = np.arctan2((data['wrm_h']-min_h)*y_sign, (data['wrm_w']-min_w)*x_sign)
        if condition is not None:
            data = data[condition(data)]
        
        # display(data.head(50))
        # g = sns.histplot(data=data, x="angle")
        g = sns.jointplot(data=data, x="wrm_speed", y="angle", kind="scatter", hue=hue, alpha=0.6)
        # g = sns.jointplot(data=data, x="worm_center_dist_x", y="worm_center_dist_y", kind="scatter", hue=hue, alpha=0.6)
        # g.set_axis_labels("distance x", "distance y")
        # g.figure.suptitle(f"distance between worm and microscope centers in each axis")

        return g.figure

In [ ]:
plot_2d_deviation_with_angle(pltr, 5, hue=None, condition=lambda df: df['wrm_speed'] > -0.3)

In [ ]:
def plot_area_vs_time(pltr:Plotter, n: int = 1, window_size: int = 15, hue=None, condition=None) -> plt.Figure:
        data = pltr.data_prep_frames(n=n)
        data["wrm_speed_avg"] = Plotter.rolling_average(data, window_size=window_size, column="wrm_speed")
        data["worm_center_dist"] = np.sqrt(
            (data["wrm_center_x"] - data["mic_center_x"]) ** 2 + (data["wrm_center_y"] - data["mic_center_y"]) ** 2
        )
        data["cycle_step"] = data["frame"] % pltr.timing_config.cycle_length
        data['angle'] = np.arctan2(data['wrm_w'], data['wrm_h'])
        # fig, ax = plt.subplots()
        mask = data["bbox_area_diff"] > 1e-3
        if condition is not None:
            mask = condition(data) & mask
        g = sns.jointplot(
            data=data[mask], x="wrm_speed_avg", y="worm_center_dist", hue=hue, kind="scatter", dropna=True)
        g.figure.suptitle(f"n = {n}, rolling window = {window_size}")
        return g.figure

In [ ]:
plot_area_vs_time(pltr, n=18, window_size=1, hue='angle')

In [ ]:

fig, ax = plt.subplots()
sns.scatterplot(data=data, x='wrm_center_x', y='wrm_center_y', hue='frame', ax=ax, alpha=0.3, linewidth=0.2)
ax.set_xlim(0)
ax.set_ylim(0)
ax.invert_yaxis()
fig.tight_layout()
plt.show()


In [ ]:
data['wrm_speed_avg'] = pltr.rolling_average(data, window_size=15, column='wrm_speed')

In [ ]:
data['marker_size'] = 0.1 + (data['bbox_area_diff'])*2

In [ ]:
data['marker_size'].describe()

In [ ]:
mask = pltr._data['wrm_w'] < 0
pltr._data.loc[mask, 'wrm_w'] = 0

mask = pltr._data['wrm_h'] < 0
pltr._data.loc[mask, 'wrm_h'] = 0

pltr._data['wrm_area'] = pltr._data['wrm_w'] * pltr._data['wrm_h']

In [ ]:
pltr._data['wrm_area'].describe()

In [ ]:
pltr._data[["wrm_w", "wrm_h", 'bbox_area_diff']].describe()

In [ ]:
(data['wrm_speed_avg'] > 2).sum()

In [ ]:
(data['bbox_area_diff'] > 0.2).sum()/len(data.index)

In [ ]:
pltr.remove_invalid_cycles(data)[["wrm_x", "wrm_y", "wrm_w", "wrm_h"]].describe()

In [ ]:
fig, ax = plt.subplots()
sns.scatterplot(data=pltr._data, x='frame', y='wrm_speed_avg', hue='bbox_area_diff', ax=ax, s=8, linewidth=0.1)
ax.set_ylim(0, 2)

In [ ]:
fig, ax = plt.subplots()
sns.scatterplot(data=pltr._data, x='wrm_speed_avg', y='bbox_area_diff', hue='wrm_area', ax=ax, s=8, linewidth=0.1)
ax.set_xlim(0, 2)

In [ ]:
data['residual'] = np.sqrt((data['wrm_center_x'] - data['mic_center_x'])**2 + (data['wrm_center_y'] - data['mic_center_y'])**2)

In [ ]:
data['residual'].describe()

In [ ]:
# data['diff_x'] = data['mic_x'].diff()
# data['diff_y'] = data['mic_y'].diff()
# sns.histplot(data=data, x='diff_y')

In [ ]:
data['cycle_step'] = data['frame']%15

In [ ]:
# fig, ax = plt.subplots()
# sns.jointplot(data=data, x='cycle_step', y='residual', ax=ax, s=8, linewidth=0.1)
sns.jointplot(data=data, x='cycle_step', y='wrm_speed', ax=ax, kind='hist')
# sns.histplot(data=data, x='cycle_step', y='residual')

In [ ]:
sns.catplot(data=data, x='cycle_step', y='wrm_speed', kind='violin')

In [ ]:
%matplotlib inline
for i in [1,2,4,8,15]:
    data = pltr.data_prep_frames(i)
    data['wrm_speed_avg'] = pltr.rolling_average(data, window_size=round(15/i), column='wrm_speed')
    # fig, ax = plt.subplots()
    mask = data['bbox_area_diff'] > 1e-3
    g = sns.jointplot(data=data[mask], x='wrm_speed_avg', y='bbox_area_diff', kind='scatter', xlim=(0, 3), dropna=True)
    g.fig.suptitle(f"i = {i}, rolling window = {round(15/i)}")


In [ ]:
plt._data.dtypes

# Run Simulation

In [ ]:
from evaluation.simulator import *
from evaluation.sim_controllers import *
from dataset.raw_dataset import ExperimentConfig
from dummy_reader import DummyReader
from evaluation.motor_controllers import *

base_path = "/Users/guycohen/Desktop/Bio-Project/"

## Run CSV Simulation

In [ ]:
exp_base_path = base_path + "Controller2"

time_config = TimingConfig.load_json(base_path + "time_config.json")
# time_config = TimingConfig.load_json(base_path + "time_config_medium_micro.json")
exp_config = ExperimentConfig.load_json(base_path + "exp_config.json")
log_config = LogConfig(
    root_folder=exp_base_path,
    save_mic_view=False,
    save_cam_view=False,
    save_err_view=False,
)

In [ ]:
from evaluation.sim_controllers.test_controller1 import TestController1
from evaluation.sim_controllers.guys_controllers import *
from evaluation.sim_controllers.csv_controller import CsvController
from evaluation.sim_controllers.optimal_controller import  OptimalController

# controller = CsvController(time_config, "/Users/guycohen/Desktop/Bio-Project/eval_corrected_movement/bboxes.csv")

# controller = TestController1(time_config, "/Users/guycohen/Desktop/Bio-Project/eval/bboxes.csv")
controller = Controller2(time_config, "/Users/guycohen/Desktop/Bio-Project/eval_corrected_movement/bboxes.csv")
# controller = OptimalController(time_config, "/Users/guycohen/Desktop/Bio-Project/eval_corrected_movement/bboxes.csv")


In [ ]:
reader = DummyReader(exp_config)

logging_controller = LoggingController(controller, log_config)
motor_controller = SineMovementController(time_config)
# motor_controller = SimpleMovementController(time_config, 0)

sim = Simulator(time_config, exp_config, logging_controller, reader=None, motor_controller=motor_controller)

In [ ]:
sim.run()
time_config.save_json(exp_base_path + "/time_config.json")
exp_config.save_json(exp_base_path + "/exp_config.json")

## Real Simulation

In [ ]:
# experiment_config = ExperimentConfig(
#     name="exp1",
#     num_frames=61200,
#     frames_per_sec=60,
#     orig_resolution=(5300, 4600),
#     px_per_mm=90,
#     init_position=(1300, 1200),
# )
# experiment_config.save_json("/Users/guycohen/Desktop/Bio-Project/exp_config.json")

In [ ]:
"""time_config = TimingConfig(
    frames_per_sec=60,
    imaging_time_ms=200,
    pred_time_ms=40,
    moving_time_ms=50,
    px_per_mm=90,
    camera_size_mm=(4, 4),
    micro_size_mm=(0.327, 0.327),
    frame_padding_value=(255, 255, 255),
)
time_config.save_json(base_path+"time_config_medium_micro.json")"""

In [ ]:

"""time_config = TimingConfig(
    frames_per_sec=60,
    imaging_time_ms=200,
    pred_time_ms=40,
    moving_time_ms=50,
    px_per_mm=90,
    camera_size_mm=(4, 4),
    micro_size_mm=(0.225, 0.225),
)

log_config = LogConfig(
    root_folder=base_path,
    save_mic_view=False,
    save_cam_view=False,
)"""

# yolo_config = YoloConfig(model_path="yolov8m-trained.pt") 

# time_config.save_json(base_path+"time_config.json")
# log_config.save_json(base_path+"csv_log_config.json")
# yolo_config.save_json(base_path++"yolo_config.json")


# frames_path = "Archive/"
# reader = FrameReader.create_from_directory(frames_path)

# time_config = TimingConfig.load_json("logs/time_config.json")
# log_config = LogConfig.load_json("logs/log_config.json")
# yolo_config = YoloConfig.load_json("logs/yolo_config.json")


In [ ]:
print(time_config.imaging_frame_num + time_config.moving_frame_num)

In [ ]:

controller = LoggingController(time_config, yolo_config, log_config)
#controller = YoloController(time_config, yolo_config)
sim = Simulator(time_config, reader, controller)

In [ ]:
sim.run()

In [ ]:
cv.destroyAllWindows()
raise Exception("Finished")

In [ ]:
from frame_reader import FrameReader

reader = FrameReader.create_from_directory("Archive/")

In [ ]:
from dataset.create.box_calculator import BoxCalculator
from dataset.create.sample_extractor import SampleExtractor

bbox_calc = BoxCalculator(reader)
extractor = SampleExtractor(bbox_calc)
extractor.create_all_samples(target_size=(400, 400), save_folder="samples")

In [ ]:
# TODO: IMPROVE
def find_boxes(image: np.ndarray) -> np.ndarray:
    # increase contrast of original image
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

    # remove gaussian noise
    smoothed = cv.GaussianBlur(image, (3, 3), 0)

    # apply threshold to image to convert it into black & white
    _, mask = cv.threshold(smoothed, 160, 255, cv.THRESH_BINARY_INV)

    # do some morphological magic to clean up noise from the mask
    kernel = np.ones((5, 5), np.uint8)
    mask = cv.morphologyEx(mask, cv.MORPH_OPEN, kernel)

    # dilate to increase all object sizes in the mask
    kernel = np.ones((3, 3), np.uint8)
    mask = cv.dilate(mask, kernel, iterations=5)

    image[mask == 0] = 255
    cv.imshow("masked", image)

    # find contours
    contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    # Populate bounding boxes
    bbox_list = []
    for c in contours:
        area = cv.contourArea(c)

        if area < 500 or area > 4000:
            continue

        box = cv.boundingRect(c)
        bbox_list.append(box)

    # Turn our bboxes into 2d ndarray
    bboxes = np.asanyarray(bbox_list)
    return bboxes